In [1]:
import torch
import torch.nn as nn
import numpy as np
from auto_LiRPA import BoundedModule, BoundedTensor
from auto_LiRPA.perturbations import PerturbationLpNorm

print(f"PyTorch version: {torch.__version__}")
try:
    import auto_LiRPA
    print(f"auto_LiRPA version: {auto_LiRPA.__version__}")
except ImportError:
    print("auto_LiRPA n'est pas installé. Veuillez l'installer avec : pip install auto-lirpa")
    exit()

PyTorch version: 2.3.1+cu121
auto_LiRPA version: 0.6.0


In [2]:
from deel.torchlip import GroupSort2, Sequential
import deel.torchlip
from collections import defaultdict

In [24]:
class GS2(nn.Module):
  def forward(self, inputs):
      # print('help')
      n = inputs.shape[-1]//2
      input_reshape = torch.reshape(inputs, (-1, n, 2))
      # apply min on first axis
      # input_min = torch.unsqueeze(torch.min(input_reshape, -1)[0], -1)
      input_min = torch.unsqueeze(torch.min(input_reshape, -1).values, -1)
      # apply max on first axis
      # input_max = torch.unsqueeze(torch.max(input_reshape, -1)[0], -1)
      input_max = torch.unsqueeze(-torch.min(-input_reshape, -1).values, -1)

      output = torch.cat([input_min, input_max],-1)
      # reshape output to have the same shape as input
      # output = torch.view(output, inputs.shape)
      output = output.view(-1,2*n)
      return output

In [25]:
model = torch.nn.Sequential(GS2())

In [26]:
dummy_input_shape = (1, 6)
dummy_input = torch.randn(dummy_input_shape)

In [27]:
dummy_input

tensor([[-0.3289,  0.0131,  0.0894,  0.0565, -1.7825,  0.1341]])

In [28]:
model(dummy_input)

tensor([[-0.3289,  0.0131,  0.0565,  0.0894, -1.7825,  0.1341]])

In [29]:
lower = dummy_input - 1
upper = dummy_input + 1

In [30]:
lirpa_model = BoundedModule(model, dummy_input)

In [31]:
pred = lirpa_model(dummy_input)

In [32]:
pred

tensor([[-0.3289,  0.0131,  0.0565,  0.0894, -1.7825,  0.1341]])

In [33]:
norm = float("inf")
ptb = PerturbationLpNorm(norm = norm, x_L=lower, x_U=upper)
bounded_input = BoundedTensor(dummy_input, ptb)

In [34]:
# Compute bounds.
lb, ub = lirpa_model.compute_bounds(x=(bounded_input,), method='IBP')
print(f'IBP bounds: lower={lb}, upper={ub}')

IBP bounds: lower=tensor([[-1.3289, -0.9869, -0.9435, -0.9106, -2.7825, -0.8659]]), upper=tensor([[ 0.6711,  1.0131,  1.0565,  1.0894, -0.7825,  1.1341]])


In [35]:
lb, ub = lirpa_model.compute_bounds(x=(bounded_input,))
print(f'CROWN bounds: lower={lb}, upper={ub}')

NotImplementedError: `bound_backward` for BoundReduceMax with perturbed maximumindexes is not implemented.

In [ ]:
# Getting the linear bound coefficients (A matrix).
required_A = defaultdict(set)
required_A[lirpa_model.output_name[0]].add(lirpa_model.input_name[0])
lb, ub, A = lirpa_model.compute_bounds(x=(bounded_input,), method='CROWN', return_A=True, needed_A_dict=required_A)
print('CROWN linear (symbolic) bounds: lA x + lbias <= f(x) <= uA x + ubias, where')
print(A[lirpa_model.output_name[0]][lirpa_model.input_name[0]])

NotImplementedError: `bound_backward` for BoundReduceMax with perturbed maximumindexes is not implemented.